In [12]:
#modules
import matplotlib.pyplot as plt
import xgboost as xgb
from matplotlib import rcParams
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
!pip install scikit_optimize
from scipy.stats import spearmanr
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
seed = 42



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import pandas as pd

train_data=pd.read_csv("/content/drive/MyDrive/novozymes-enzyme-stability-prediction/train.csv")

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31104 non-null  float64
 3   data_source       28043 non-null  object 
 4   tm                31390 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


#data preprocessing

In [15]:
#feature-1 Length of the protien 
!pip3 install protlearn
import protlearn as pl 
from protlearn.features import length, entropy, aaindex1, atc, aac

# Why Protlearn Module ?
"""
A technique for extracting features from protein sequences is called protolearn. 
It is a free Python tool that enables users to effectively extract amino acid sequence features from proteins or peptides. 
These features may then be utilized for a range of downstream machine learning tasks. 
Currently, protlearn consists of three primary modules: 

dimension reduction and feature extraction during preprocessing 
The Preprocessing area enables the user to prepare the datasets in accordance with certain requirements, such as removing sequences that include unnatural amino acids or changing alphabetical strings into integers. 
The dataset's amino acid sequence features, such as amino acid composition or AAIndex-based physicochemical attributes, can then be computed using the Feature Extraction section. 
Protlearn presently offers 21 such functions in total.
The dimensionality of the computed features can be reduced using feature selection techniques, which lessens redundant computation and lessens the computational load on the classifiers.
 

From my research from the protien sequence the important elements we can extract can be done by using the below functions:

Functions Used:

1. Amino acid composition: (aac)
This function returns the frequency of amino acids for each sequence in the dataset. aac is calculated as follows:
𝑎𝑎𝑐(𝑖) = 𝑛𝑖/N
    
protlearn.features.aac(X, *, method='relative', remove_zero_cols=False, start=1, ˓→end=None)
 where i denotes the 20 amino acid residues, ni is the frequency of each residue i, and N is the total number of residues in the sequence.


2. AAIndex1-based physicochemical properties.
AAindex1 ver.9.2 (release Feb, 2017) is a set of 20 numerical values representing various physicochemical and bio- logical properties of amino acids. Currently, it contains 566 indices, of which 553 contain no NaNs. The indices will be collected for each amino acid in the sequence, then averaged across the sequence.
aaindex1 is calculated as follows:

∑︀𝑁𝑛=1 (𝐴𝐴𝑖𝑛𝑑𝑒𝑥𝑖(𝑎𝑎𝑛) *𝑎𝑎𝑖𝑛𝑑𝑒𝑥1(𝑖))/ 𝑁

where i denotes the 566 AAIndex1 indices, aan denotes the amino acid at position n, and N is the total number of residues in the sequence.

3. protlearn.features.atc(X, *, method='relative', start=1, end=None)
Atomic and bond composition.

This function returns the sum of atomic and bond compositions for each amino acid sequence. The atomic features are comprised of five atoms (C, H, N, O, and S), and the bond features are comprised of total bonds, single bonds, and double bonds.
atc is calculated as follows:
𝑎𝑡𝑜𝑚𝑠(𝑖) = 𝑛𝑖/𝑁
"""


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


"\nA technique for extracting features from protein sequences is called protolearn. \nIt is a free Python tool that enables users to effectively extract amino acid sequence features from proteins or peptides. \nThese features may then be utilized for a range of downstream machine learning tasks. \nCurrently, protlearn consists of three primary modules: \n\ndimension reduction and feature extraction during preprocessing \nThe Preprocessing area enables the user to prepare the datasets in accordance with certain requirements, such as removing sequences that include unnatural amino acids or changing alphabetical strings into integers. \nThe dataset's amino acid sequence features, such as amino acid composition or AAIndex-based physicochemical attributes, can then be computed using the Feature Extraction section. \nProtlearn presently offers 21 such functions in total.\nThe dimensionality of the computed features can be reduced using feature selection techniques, which lessens redundant co

In [69]:
def data_preprocessing(train_data):
  import pandas as pd 
  def length_of_protien(df):
    protien=[]
    for i in df["protein_sequence"]:
      protien.append(len(i))
    return protien 
    #feature - 2 Entropy 

  def entropy_finder(df):
    entropy=[]
    for i in df["protein_sequence"]:
      entropy.append(i)
    return entropy 
  def length_of_protien(df):
    protien=[]
    for i in df["protein_sequence"]:
      protien.append(len(i))
    return protien 
  #feature - 3
  atoms, bonds = atc(train_data['protein_sequence'].tolist())
  comp, aa = aac(train_data['protein_sequence'].tolist())
  # Composition in the protien sequences 

  # Atoms in the protien sequences 

  # Bonds in the protien sequencs 

  def Form_a_data(keys,values):
    sample_dict = dict()
    dict_keys = keys
    for i in range(len(dict_keys)):
        sample_dict[dict_keys[i]] = [v[i] for v in values]
    return pd.DataFrame(sample_dict)
  compostion_data=Form_a_data([*aa],comp)
  atom_Data=Form_a_data(['Carbon','Hydrogen','Nitrogen','Oxygen','Sulphur'],atoms)
  bonds_data=Form_a_data(['total_bonds','single','double'],bonds)
  train_data["protien_length"]=length_of_protien(train_data)

  train_data["entropy"]=entropy_finder(train_data)

  train_data["protien_length"]=length_of_protien(train_data)

  main_data=pd.concat([train_data,compostion_data,atom_Data,bonds_data],axis=1)
  main_data.drop(["protein_sequence","data_source","seq_id"],axis=1,inplace=True)
  main_data.drop(["entropy"],axis=1,inplace=True)
  return main_data
main_data=data_preprocessing(train_data)
  

In [70]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pH              31104 non-null  float64
 1   tm              31390 non-null  float64
 2   protien_length  31390 non-null  int64  
 3   A               31390 non-null  float64
 4   C               31390 non-null  float64
 5   D               31390 non-null  float64
 6   E               31390 non-null  float64
 7   F               31390 non-null  float64
 8   G               31390 non-null  float64
 9   H               31390 non-null  float64
 10  I               31390 non-null  float64
 11  K               31390 non-null  float64
 12  L               31390 non-null  float64
 13  M               31390 non-null  float64
 14  N               31390 non-null  float64
 15  P               31390 non-null  float64
 16  Q               31390 non-null  float64
 17  R               31390 non-null 

In [72]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pH              31104 non-null  float64
 1   tm              31390 non-null  float64
 2   protien_length  31390 non-null  int64  
 3   A               31390 non-null  float64
 4   C               31390 non-null  float64
 5   D               31390 non-null  float64
 6   E               31390 non-null  float64
 7   F               31390 non-null  float64
 8   G               31390 non-null  float64
 9   H               31390 non-null  float64
 10  I               31390 non-null  float64
 11  K               31390 non-null  float64
 12  L               31390 non-null  float64
 13  M               31390 non-null  float64
 14  N               31390 non-null  float64
 15  P               31390 non-null  float64
 16  Q               31390 non-null  float64
 17  R               31390 non-null 

In [73]:
def train_the_model(main_data1):
  import xgboost as xgb
  from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
  from sklearn.model_selection import train_test_split

  y=main_data1["tm"]
  main_data1.drop(["tm"],axis=1,inplace=True)
  X=main_data1

  from sklearn.model_selection import train_test_split

  import numpy as np

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

  search_spaces = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}
  def trainer(X_train,y_train,X_test,y_test,search_spaces):

      from sklearn.model_selection import RandomizedSearchCV

      xgbr = xgb.XGBRegressor(seed = 20)
      clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=search_spaces,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=1)
      
      clf.fit(X_train, y_train)
      print("Best parameters:", clf.best_params_)
      print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

      y_pred = pred = clf.predict(X_test) 

      trials = Trials()
      return y_test,y_pred
  y_test,y_pred=trainer(X_train,y_train,X_test,y_test,search_spaces)
  return y_test,y_pred
main_data1=main_data
original_tester,predicted_tester = train_the_model(main_data1)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[17:28:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:28:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:28:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:28:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:28:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:28:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:28:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:28:56] WARNING: /workspace/src/objective/regr

# XG BOOST REGRESSOR :

A class of ensemble machine learning methods known as gradient boosting can be applied to classification or regression predictive modeling issues. 

Decision tree models are used to build ensembles. In order to repair the prediction mistakes caused by earlier models, trees are added one at a time to the ensemble and fitted. Boosting is a term used to describe this kind of ensemble machine learning model. 

Any arbitrary differentiable loss function and the gradient descent optimization procedure are used to fit the models. Gradient boosting gets its name from this because as the model is fitted, it minimizes the loss gradient, much like a neural network.

Execution speed and model performance are the two primary benefits of using XGBoost. 

On structured or tabular datasets for classification and regression predictive modeling issues, XGBoost is dominant. The fact that it is the preferred method for contest winners on the Kaggle platform for competitive data science serves as proof. 

The following are the most typical configurations for hyperparameters: 

The number of trees in the ensemble, which is frequently raised until no additional gains are observed, is called n estimators. 

max depth: Each tree's maximum depth, typically between 1 and 10. 

eta: The learning rate applied to weight each model, 

frequently set to tiny values like 0.3, 0.1, or 0.01.
 
the number of samples (rows) in each tree that is set to a value between 0 and 1.

Colsample_bytree: Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 to use all features.



In [80]:
train_data=pd.read_csv("/content/drive/MyDrive/novozymes-enzyme-stability-prediction/train.csv")
main_data=data_preprocessing(train_data)
def final_model(main_data1):
  y=main_data1["tm"]
  main_data1.drop(["tm"],axis=1,inplace=True)
  X=main_data1

  from sklearn.model_selection import train_test_split

  import numpy as np

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
  final_model = xgb.XGBRegressor(seed = 20, n_estimators = 1000, max_depth = 10, learning_rate = 0.01, colsample_bytree = 0.5, colsample_bylevel =0.7)
  final_model.fit(X_train, y_train)
  y_pred = pred = final_model.predict(X_test) 
  return final_model 
regressor_model=final_model(main_data)

[19:40:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [74]:
def evaluvation_metrics(y_test,y_pred):
  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import r2_score
  import numpy as np 
  print("*"*20)
  print("*"*8," Evaluvation Scores ","*"*8)
  print("\nMean Absolute Error : ",mean_absolute_error(y_test,y_pred))
  print("\nMean Squared Error : ",mean_squared_error(y_test,y_pred))
  print("\nRoot Mean Squared Error : ",np.sqrt(mean_squared_error(y_test,y_pred)))
  print("\nR Squared Score : ",r2_score(y_test,y_pred))
evaluvation_metrics(original_tester,predicted_tester)

********************
********  Evaluvation Scores  ********

Mean Absolute Error :  6.044688573561619

Mean Squared Error :  73.4385597517905

Root Mean Squared Error :  8.569630082552601

R Squared Score :  0.6083250378066907


#Testing using the test.csv file

In [75]:
import pandas as pd 
test_data=pd.read_csv("/content/drive/MyDrive/novozymes-enzyme-stability-prediction/test.csv")
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   seq_id            2413 non-null   int64 
 1   protein_sequence  2413 non-null   object
 2   pH                2413 non-null   int64 
 3   data_source       2413 non-null   object
dtypes: int64(2), object(2)
memory usage: 75.5+ KB


In [76]:
#Now access data preprocessing unit using (data_preprocessing function)
test_data = data_preprocessing(test_data)

In [81]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2413 entries, 0 to 2412
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pH              2413 non-null   int64  
 1   protien_length  2413 non-null   int64  
 2   A               2413 non-null   float64
 3   C               2413 non-null   float64
 4   D               2413 non-null   float64
 5   E               2413 non-null   float64
 6   F               2413 non-null   float64
 7   G               2413 non-null   float64
 8   H               2413 non-null   float64
 9   I               2413 non-null   float64
 10  K               2413 non-null   float64
 11  L               2413 non-null   float64
 12  M               2413 non-null   float64
 13  N               2413 non-null   float64
 14  P               2413 non-null   float64
 15  Q               2413 non-null   float64
 16  R               2413 non-null   float64
 17  S               2413 non-null   f

In [84]:
#Now use the model 
y_pred=regressor_model.predict(test_data)
